# 6 - Summarization

If you think about it, text summarization requires a range of abilities, such as understanding long passages, reasoning about the contents, and producing fluent text that incorporates the main topics from the original document. Moreover, accurately summarizing a news article is very different from summarizing a legal contract, so being able to do so requires a sophisticated degree of domain generalization. For these reasons, text summarization is a difficult task for neural language models, including transformers.

Despite these challenges, text summarization offers the prospect for domain experts to significantly speed up their workflows and is used by enterprises to condense internal knowledge, summarize contracts, automatically generate content for social media releases, and more.

In this chapter we will build our own encoder-decoder model to condense dialogues between several people into a crisp summary. 

## 6.1 - The CNN/DailyMail Dataset

Before we dive into the summarization process, let's begin by taking a look at one of the canonical datasets for summarization: the CNN/DailyMail corpus. This dataset consists of around 300000 pairs of news articles and their corresponding summaries, composed from the bullet points that CNN and the DailyMail attach to their articles. 

<span style="color:blue">An important aspect of the dataset is that the summaries are <b>abstractive</b> and not <b>extractive</b>, which means that they consist of new sentences instead of simple excerpts.</span> [**The dataset is available on the Hub**](https://huggingface.co/datasets/cnn_dailymail); we'll use version 3.0.0, which is a nonanonymized version set up for summarization. 

In [ ]:
from datasets import load_dataset

dataset = load_dataset("cnn_dailymail", version="3.0.0")
print(f"Features: {dataset['train'].column_names}")

The dataset has three columns: `article`, which contains the news articles, `highlights` with the summarise, and `id` to uniquely identify each article. Let's look at an excerpt from an article:

In [ ]:
sample = dataset["train"][1]
print(f"""Article (excerpt of 500 characters, total length: {len(sample["article"])}):""")
print(sample["article"][:500])
print(f'\nSummary (length: {len(sample["highlights"])}):')
print(sample["highlights"])

We see that the articles can be very long compared to the target summary; in this particular case the difference is 17-fold. Long articles pose a challenge to most transformer models since the context size is usually limited to 1000 tokens or so, which is equivalent to a few paragraphs of text. The standard, yet crude way to deal with this for summarization is to simply truncate the texts beyond the model's context size. Obviously there could be important information for the summary toward the end of the text, but for now we need to live with this limitation of the model architectures.



## 6.2 - Text summarization pipelines

Let's see how a few of the most popular transformer models for summarization perform by first looking qualitatively at the outputs for the preceding example. Although the model architectures we will be exploring have varying maximum input sizes, let's restrict the input text to 2000 characters to have the same input for all models and thus make the outputs more comparable:



In [ ]:
sample_text = dataset["train"][1]["article"][:2000]
# We'll collect the generated summaries of each model in a dictionary
summaries = {}

A convention in summarization is to separate the summary sentences by a newline. We could add a newline token after each full stop, but this simple heuristic would fail for strings like "U.S" or "U.N". The Natural language toolkit (NLTK) package includes a more sophisticated algorithm that can differentiate the end of a sentence from punctuation that occurs in abbrevisations:



In [ ]:
import nltk
from nltk.tokenize import sent_tokenize
nltk.download("punkt")
string = "The U.S. are a country. The U.N. is an organization."
sent_tokenize(string)

## 6.3 - Summarization baseline

A common baseline for summarizing news articles is to simply take the first three sentences of the article. With NLTK's sentence tokenizer, we can easily implement such a baseline:



In [ ]:
def three_sentence_summary(text):
    return "\n".join(sent_tokenize(text)[:3])

summaries["baseline"] = three_sentence_summary(sample_text)

### 6.3.1 - GPT-2

We have already seen in Chapter 5 how GPT-2 ([Radford et al., 2019](https://openai.com/blog/better-language-models/)) can generate text given some prompt. One of the model's surprising features is that we can also use it to generate summaries by simply appending "TL;DR" at the end of the input text. This expression is often used on platforms like Reddit to indicate a short version of a long post. 

We will start our summarization experiment by re-creating the procedure of the original paper with the `pipeline()` function from 🤗 Transformers. We create a text generation pipeline and load the GPT-2 model:

In [ ]:
from transformers import pipeline, set_seed
set_seed(42)
pipe = pipeline("text-generation", model="gpt2") # 117M parameters
gpt2_query = sample_text + "\nTL;DR:\n"
pipe_out = pipe(gpt2_query, max_length=512, clean_up_tokenization_spaces=True)
summaries["gpt2"] = "\n".join(
sent_tokenize(pipe_out[0]["generated_text"][len(gpt2_query) :]))

Here we just store the summaries of the generated text by slicing off the input query and keep the result in a Python dictionary for later comparison.

### 6.3.2 - T5

Next let's try the T5 transformer ([Raffel et al., 2019](https://arxiv.org/abs/1910.10683)). As we saw in Chpater 3, the developers of this model performed a comprehensive study of transfer learning in NLP and found they could create a universal transformer architecture by formulating all tasks as text-to-text tasks. The T5 checkpoints are trained on a mixture of unsupervised data (to reconstruct masked words) and supervised data for several tasks, including summarization. These checkpoints can thus be directly used to perform summarization without fine-tuning by using the same prompts used during pretraining. In this framework, the input format for the model to summarize a document is `"summarize:" <ARTICLE>`, and for translation it looks like `"translate English to German:" <TEXT>`. This maskes T5 extremely versatile and allows us to solve many tasks with a single model.

We can directly load T5 for summarization with the `pipeline()` function, which also takes care of formatting the inputs in the text-to-text format so we don't need to prepend them with `"summarize:"`

<img src="images/t5_examples.png" title="" alt="" width="700" data-align="center">

In [ ]:
pipe = pipeline("summarization", model="t5-small") # We could also try t5-base, with 220M parameters
pipe_out = pipe(sample_text)
summaries["t5"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))

### 6.3.3 - BART

BART ([Lewis et al., 2019](https://arxiv.org/abs/1910.13461)) also uses an encoder-decoder architecture and is trained to reconstruct corrupted inputs. It combines the pretraining schemes of BERT and GPT-2. We'll use the `facebook/bart-base-cnn` checkpoint, which has been specifically fine-tuned on the CNN/DailyMail dataset:

In [ ]:
pipe = pipeline("summarization", model="facebook/bart-base-cnn") # 140M parameters
pipe_out = pipe(sample_text)
summaries["bart"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))

### 6.3.4 - PEGASUS

PEGASUS ([Zhang et al., 2019](https://arxiv.org/abs/1912.08777)) is also an encoder-decoder transformer. Its pretraining objective is to is to predict masked sentences in multisentence texts. The authors argue that the closer the pretraining objective is to the downstream task, the more effective it is. With the aim of finding a pretraining objective that is closer to summarization than general language modeling, they automatically identified, in a very large corpus, sentences containing most of the content of their surrounding paragraphs (using summarization evaluation metrics as a heuristic for content overlap) and pretrained the PEGASUS model to reconstruct these sentences, thereby obtaining a state-of-the-art model for text summarization.

<img src="images/pegasus_architecture.png" title="" alt="" width="700" data-align="center">

In [ ]:
# Note on size: Pegasus is 568M parameters, which is considerably larger than BART-base and T5-base
pipe = pipeline("summarization", model="google/pegasus-cnn_dailymail")
pipe_out = pipe(sample_text)
summaries["pegasus"] = pipe_out[0]["summary_text"].replace(" .<n>", ".\n")

## 6.4 - Comparing different summaries

Now that we have generated summaries with four different models, let's compare the results. Keep in mind that one model has not been trained on the dataset at all (GPT-2), one model has been fine-tuned on this task among others (T5), and two models have exclusively been fine-tuned on this task (BART and PEGASUS). Let's have a look at the summaries these models have generated:



In [ ]:
print("GROUND TRUTH")
print(dataset["train"][1]["highlights"])
print("")
for model_name in summaries:
print(model_name.upper())
print(summaries[model_name])
print("")

## 6.5 - Measuring the quality of generated text

Good evaluation metrics are important, since we use them to measure the performance of models not only when we train them but also later, in production. If we have bad metrics we might be blind to model degradation, and if they are misaligned with the business goals we might not create any value.

Measuring performance on a text generation task is not as easy with standard classification tasks such as sentiment analysis or named entity recognition. Take the example of translation; given a sentence like "I love dogs!" in English and translating it to Spanish there can be multiple valid possibilities, like "¡Me encantan los perros!" or "¡Me gustan los perros!"

<span style="color:blue">Simply checking for an exact match to a reference translation is not optimal; even humans would fare badly on such a metric because we all write text slightly differently from each other (and even from ourselves, depending on the time of the day or year!). Fortunately, there are alternatives. </span>

Two of the most common metrics used to evaluate generated text are **BLEU** and **ROUGE**. Let's take a look at how they are defined.

### 6.5.1 - BLEU

The idea of BLEU ([Papineni et al., 2002](https://dl.acm.org/doi/10.3115/1073083.1073135)) is simple: instead of looking at how many of the tokens in the generated texts are perfectly aligned with the reference text tokens, we look at words or $n$-grams. BLEU is a precision-based metric, which means that when we compare two texts we count the number of words in the generation that occur in the reference and divide it by the length of the reference.

However, there is an issue with this vanilla precision. Assume the generated text just repeats the same word over and over again, and this word also appears in the reference. If it is repeated as many times as the length of the reference text, then we get perfect precision! 

For this reason, the authors of the BLEU paper introduced a slight modification: a word is only counted as many times as it occurs in the reference. To illustrate this point, suppose we have the reference text "the cat is on the mat" and the generated text is "the the the the the the". From this simple example, we can calculate the precision values as follows:

$$
p_{vanilla} = \frac{6}{6}
$$

$$
p_{mod} = \frac{2}{6}
$$

As we can see, that simple correction has pruced a much more reasonable value. Now let's extend this by not only looking at single words, but $n$-grams as well. Let's assume we have one generated sentence, $snt$, that we want to compare against a reference sentence $snt'$. We extract all possible $n$-grams of degree $n$ and do the accounting to get the precision $p_{n}$:

$$
p_{n} = \frac{\sum_{n\text{-gram} \ \in \ snt} \text{Count}_{clip}(n\text{-gram})}{\sum_{n\text{-gram} \ \in \ snt'} \text{Count}(n\text{-gram})}
$$

In order to avoid rewarding repetitive generations, the count in the numerator is clipped. What this means is that the occurrence count of an $n$-gram is capped at how many times it appears in the reference sentence. Also note that the definition of a sentence is not very strict in this equation, and if you had a generated text spanning multiple sentences you would treat is as one sentence.

In general, we have more than one sample in the test set we want to evaluate, so we need to slightly extend the equation by summing over all samples in the corpus $C$ (we are assumming that $C$ contains both the original sentences and the generated ones):

$$
p_{n} = \frac{\sum_{snt \ \in \ C}\sum_{n\text{-gram} \ \in \ snt} \text{Count}_{clip}(n\text{-gram})}{\sum_{snt' \ \in \ C} \sum_{n\text{-gram} \ \in \ snt'} \text{Count}(n\text{-gram})}
$$

We are almost there. Since we are not looking at recall, all generated sequences that are short but precise have a benefit compared to sentences that are longer. Therefore, the precision score favors short generations. To compensate for that, authors of BLEU introduced an additional term, the brevity ($BR$) penalty:

$$
BR = \text{min} \left( 1, e^{1-l_{ref}/l_{gen}} \right)
$$

By taking the minimum, we ensure that this penalty never exceeds 1 and the exponential term becomes exponentially small when the length of the generated text $l_{gen}$ is smaller than the reference text $l_{ref}$.

At this point you may ask, why don't we just use something like an F1-score to account for recall as well? The answer is that often in translation datasets there are multiple reference sentences instead of just one, so if we also measured recall we would  incentivize translations that used all the words from all the references. Therefore, it is preferable to look for high precision in the translation and make sure the translation and reference have a similar length.

Finally, we can put everything together and get the equation for the BLEU score, where the last term is the geometric mean of the modified precision up to $n$-gram $N$. 

$$
\text{BLEU-}N = BR \times \left( \prod^{N}_{n=1} p_{n}\right)
$$

In practice, the BLEU-4 score is often reported. Howver, you can probably already see that this metric has many limitations; for instance, it doesn't take synonyms into account, and many steps in the derivation seem like ad hoc and rather fragile heuristics  You can find a wonderful exposition of BLEU's flaws in Rachel Tatman's blog post ["Evaluating Text Output in NLP: BLEU at Your Own Risk"](https://towardsdatascience.com/evaluating-text-output-in-nlp-bleu-at-your-own-risk-e8609665a213).

In general, the field of text generation is still looking for better evaluation metrics, and finding ways to overcome the limits of metrics like BLEU is an active area of research. Another weakness of the BLEU metric is that it expects the texto already be tokenized. This can lead to varying results if the exact same method for text tokenization is not used. The SacreBLEU metric addresses this issue by internalizing the tokenization step; for this reason, is the prefered metric for benchmarking.

In [ ]:
from datasets import load_metric
bleu_metric = load_metric("sacrebleu")

The `bleu_metric` object is an instance of the `Metric` class, and works like an aggregator: you can add single instances with `add()` or whole batches via `add_batch()`. Once you have added all the samples you need to evaluate, you then call `compute()` and the metric is calculated. This returns a dictionary with several values, such as the precision for each $n$-gram, the length penalty, as well as the final BLEU score. Let's look at the example from before:



In [ ]:
import pandas as pd
import numpy as np

bleu_metric.add(prediction="the the the the the the", reference=["the cat is on the mat"])

results = bleu_metric.compute(smooth_method="floor", smooth_value=0)
results["precisions"] = [np.round(p, 2) for p in results["precisions"]]
pd.DataFrame.from_dict(results, orient="index", columns=["Value"])

----

**Note:** The BLEU score also works if there are multiple reference translations. This is why `reference` is passed as a list. To make the metric smoother for zero counts in the $n$-grams, BLEU integrates methods to modify the precision calculation. One method is to add a constant to the numerator. That way, a missing $n$-gram does not cause the score to automatically go to zero. For the purpose of explaining the values, we turn it off by setting `smooth_value = 0`

----

We can see the precision of the 1-gram is ineed 2/6, whereas the precisions for the 2/3/4-grams are all 0 (for more information about the individual metrics, like `counts` and `bp`, see the [SacreBLEU repository](https://github.com/mjpost/sacrebleu)). This means the geometric mean is zero, thus also the BLEU score. Let's look at another example where the prediciton is almost correct:

In [ ]:
bleu_metric.add(prediction="the cat is on mat", reference=["the cat is on the mat"])
results = bleu_metric.compute(smooth_method="floor", smooth_value=0)
results["precisions"] = [np.round(p, 2) for p in results["precisions"]]
pd.DataFrame.from_dict(results, orient="index", columns=["Value"])

We observe that the precision scores are much better. The 1-grams in the prediciton all match, and only in the precision scores do we see that something is off. For the 4-gram there are only two candidates in the predicted text, i.e., `["the", "cat", "is", "on"]` and `["cat", "is", "on", "mat"]`, where the last one doesn't match, hence the precision of 0.5.

<span style="color:blue">The BLEU score is widely used for evaluating text, especially in machine translation, since precise translations are usually favored over translations that include all possible and appropriate words. There are other applications, such as summarization, where the situation is diferent. There we want all the important information in the generated text, so we favour high recall. This is where the ROUGE score is usually used.</span>



### 6.5.2 - ROUGE

The ROUGE score ([C-Y. Lin, 2004](https://aclanthology.org/W04-1013.pdf)) was specifically developed for applications like summarization where high recall is more important than just precision. The approach is very similar to the BLEU score in that we look at different $n$-grams and compare their occurrences in the generated text and the reference texts. 

* With ROUGE, we check how many $n$-grams in the reference text also occur in the generated text.
* With BLEU, we check many $n$-grams in the generated text appear in the reference.

Given their similar definitions, we can reuse the precision formula with the minor modification that we count the (unclipped) occurrence of reference $n$-grams in the generated text in the numerator:

$$
\text{ROUGE-}N = \frac{\sum_{snt' \ \in \ C} \ \sum_{n\text{-gram} \ \in \ snt'} \text{Count}_{\text{match}}(n\text{-gram})}{\sum_{snt' \ \in \ C} \ \sum_{n\text{-gram} \ \in \ snt'} \text{Count}(n\text{-gram})}
$$

This was the original proposal for ROUGE. Subsequently, researchers have found that fully removing precision can have strong negative effects. Going back to the BLEU formula without the clipped counting, we can measure precision as well, and we can then combine both precision and recall ROUGE score in the harmonic mean to get an F1-score. The score is the metric that is nowadays commonly reported for ROUGE.

There is a separate score in ROUGE to measure the longest common substring (LCS), called ROUGE-L. The LCS can be calculated for any pair of strings. For example, the LCS for "abab" and "abc" would be "ab", and its lenght would be 2. If we want to compare this value between two samples we need to somehow normalize it because otherwise a longer text would be at an advantage. To achieve this, the inventor of ROUGE came up with an F-score-like scheme where the LCS is normalized with the length of the reference and generated text, then the two normalized scores are mixed together:

$$
R_{LCS} = \frac{LCS(X,Y)}{m} 
$$


$$
P_{LCS} = \frac{LCS(X,Y)}{n}
$$

$$
F_{LCS} = \frac{(1 + \beta^{2})R_{LCS}P_{LCS}}{R_{LCS} + \beta P_{LCS}}, \text{where} \beta = P_{LCS} / R_{LCS}
$$

**Note:** My assumption is that $m$ is the legnth of the reference text and $n$ refers to the length of the predicted text.

This way, the LCS score is properly normalized and can be compared across samples. In the 🤗 Datasets implementation, two variations of ROUGE are calculated: one calcuates the score per sentence and averages it for the summaries (ROUGE-L), and the other calcuates it directly over the whole summary (ROUGE-Lsum).

We have already generated a set of summaries and now we have a metric to compare the summaries systematically. Let's apply the ROUGE score to all the summaries generated by the models:

In [ ]:
reference = dataset["train"][1]["highlights"]
records = []
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

for model_name in summaries:
    rouge_metric.add(prediction=summaries[model_name], reference=reference)
    score = rouge_metric.compute()
    rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
    records.append(rouge_dict)
pd.DataFrame.from_records(records, index=summaries.keys())

----

**Note:** The ROUGE metric in the 🤗 Datasets library also calculates confidence intervals (by default, the 5th and 95th percentiles). The average value is stored in the attribute `mid` and the internval can be retrieved with `low` and `high`.

----

These results are obviously not very reliable as we only looked at a single sample, but we can compare the quality of the summary for that one example. 

<span style="color:red"><b>TODO: Rellenar una vez ejecutado y poner conclusiones</b></span>

## 6.6 - Evaluating PEGASUS on the CNN/DailyMail Dataset

We now have all the pieces in place to evaluate the model properly:

* We have a dataset with a test set from CNN/DailyMail
* We have a metric with ROUGE
* We have a summarization model

Let's first evaluate the performance of the three-sentence baseline

In [ ]:
def evaluate_summaries_baseline(dataset, metric, column_text="article", column_summary="highlights"):
    summaries = [three_sentence_summary(text) for text in dataset[column_text]]
    metric.add_batch(predictions=summaries, references=dataset[column_summary])
    score = metric.compute()
    return score

Now we'll apply the function to a subset of the data. Since the test fraction of the CNN/DailyMail dataset consists of roughly 10000 samples, generating summaries for all these articles takes a lot of time. Recall that every generated token requires a forward pass through the model; generating just 100 tokens for each sample will thus require 1 million forward passes, and if we use beam search, this number is multiplied by the number of beams. 

For the purpose of keeping the calculations relatively fast, we'll subsample the test set and run the evaluation on 100 samples instead. This should give us a much more stable estimation while completing in less than one hour on a single GPU for the PEGASUS model.

In [ ]:
test_sampled = dataset["test"].shuffle(seed=42).select(range(100))

score = evaluate_summaries_baseline(test_sampled, rouge_metric)
rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
pd.DataFrame.from_dict(rouge_dict, orient="index", columns=["baseline"]).T

Now let’s implement the same evaluation function for evaluating the PEGASUS model:

In [ ]:
from tqdm import tqdm
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

def chunks(list_of_elements, batch_size):
    """Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]
        
def evaluate_summaries_pegasus(dataset, metric, model, tokenizer, batch_size=16, device=device, column_text="article", column_summary="highlights"):
    
    article_batches = list(chunks(dataset[column_text], batch_size))
    target_batches = list(chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(zip(article_batches, target_batches), total=len(article_batches)):
        inputs = tokenizer(article_batch, max_length=1024, truncation=True, padding="max_length", return_tensors="pt")
        summaries = model.generate(input_ids=inputs["input_ids"].to(device), attention_mask=inputs["attention_mask"].to(device), length_penalty=0.8, num_beams=8, max_length=128)
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, clean_up_tokenization_spaces=True) for s in summaries]
        decoded_summaries = [d.replace("<n>", " ") for d in decoded_summaries]
        metric.add_batch(predictions=decoded_summaries, references=target_batch)
        
    score = metric.compute()
    return score

Let's unpack this evaluation code a bit. First we split the dataset into smaller batches that we can process simultaneously. Then for each batch we tokenize the input articles and feed them to the `generate()` function to produce the summaries using beam search. We use the same generation parameters as proposed in the paper. The `length_penalty` parameter ensures that the model does not generate sequences that are too long. Finally, we decode the generated texts, replace the `<n>` token, and add the decoded texts with the references to the metric. At the end, we compute and return the ROUGE scores. Let's now load the model again with the `AutoModelForSeq2SeqLM` class, used for seq2seq generation tasks, and evaluate it:

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_ckpt = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)
score = evaluate_summaries_pegasus(test_sampled, rouge_metric, model, tokenizer, batch_size=8)
rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
pd.DataFrame(rouge_dict, index=["pegasus"])

Since ROUGE and BLEU correlate better with human judgement than loss or accuracy, we should focus on them and carefully explore and choose the decoding strategy when building text generation models. These metrics are far from perfect, however, and one should always consider human judgments as well.

Now that we are equipped with an evaluation function, it's time to train our own model for summarization.

## 6.7 - Training a summarization model

We have worked through a lot of details on text summarization and evaluation, so let's put this to use to train a custom text summarization model! For our application, we'll use the [SAMSum dataset](https://huggingface.co/datasets/samsum) developed by Smasung, which consists of a collection of dialogues along with brief summaries. In an enterprise setting, these dialogues might represent the interactions between a customer and the support center, so generating accurate summaries can help improve customer service and detect common patterns among customer requests. Let's load it and look at an example:

In [ ]:
dataset_samsum = load_dataset("samsum")
split_lengths = [len(dataset_samsum[split])for split in dataset_samsum]

print(f"Split lengths: {split_lengths}")
print(f"Features: {dataset_samsum['train'].column_names}")
print("\nDialogue:")
print(dataset_samsum["test"][0]["dialogue"])
print("\nSummary:")
print(dataset_samsum["test"][0]["summary"])

The dialogues look like what you would expect from a chat via SMS or WhatsApp, including emojis and placeholders for GIFs. The `dialogue` field contains the full text and the `summary` the summarized dialogue. Could a model that was fine-tuned on the CNN/DailyMail dataset deal with that? Let's find out!

### 6.7.1 - Evaluating PEGASUS on SAMSum

First we'll run the same summarization pipeline with PEGASUS to see what the output looks like. We can reuse the code we used for the CNN/DailyMail summary generation:

In [ ]:
pipe_out = pipe(dataset_samsum["test"][0]["dialogue"])
print("Summary:")
print(pipe_out[0]["summary_text"].replace(" .<n>", ".\n"))

We can see that the model mostly tries to summarize by extracting the key sentences from the dialogue. This probably worked relatively well on the CNN/DailyMail dataset, but the summaries in SAMSum are more abstract. Let’s confirm this by running the full ROUGE evaluation on the test set:

In [ ]:
score = evaluate_summaries_pegasus(dataset_samsum["test"], rouge_metric, model, tokenizer, column_text="dialogue", column_summary="summary", batch_size=8)
rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
pd.DataFrame(rouge_dict, index=["pegasus"])

Well, the results aren’t great, but this is not unexpected since we’ve moved quite a bit away from the CNN/DailyMail data distribution. Nevertheless, setting up the evaluation pipeline before training has two advantages: we can directly measure the success of training with the metric and we have a good baseline.

Fine-tuning the model on our dataset should result in an immediate improvement in the ROUGE metric, and if that is not the case we’ll know something is wrong with our training loop.

### 6.7.2 - Fine-tuning PEGASUS

Before we process the data for training, let's have a quick look at the length distribution of the input and outputs:

In [ ]:
d_len = [len(tokenizer.encode(s)) for s in dataset_samsum["train"]["dialogue"]]
s_len = [len(tokenizer.encode(s)) for s in dataset_samsum["train"]["summary"]]

fig, axes = plt.subplots(1, 2, figsize=(10, 3.5), sharey=True)
axes[0].hist(d_len, bins=20, color="C0", edgecolor="C0")
axes[0].set_title("Dialogue Token Length")
axes[0].set_xlabel("Length")
axes[0].set_ylabel("Count")
axes[1].hist(s_len, bins=20, color="C0", edgecolor="C0")
axes[1].set_title("Summary Token Length")
axes[1].set_xlabel("Length")
plt.tight_layout()
plt.show()

We see that most dialogues are much shorter than the CNN/DailyMail articles, with 100–200 tokens per dialogue. Similarly, the summaries are much shorter, with around 20–40 tokens (the average length of a tweet).

Let’s keep those observations in mind as we build the data collator for the `Trainer`. First we need to tokenize the dataset. For now, we’ll set the maximum lengths to 1024 and 128 for the dialogues and summaries, respectively:

In [ ]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch["dialogue"], max_length=1024, truncation=True)
    
    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch["summary"], max_length=128, truncation=True)
        
    return {
        "input_ids": input_encodings["input_ids"], 
        "attention_mask": input_encodings["attention_mask"], 
        "labels": target_encodings["input_ids"]
    }

dataset_samsum_pt = dataset_samsum.map(convert_examples_to_features, batched=True)
columns = ["input_ids", "labels", "attention_mask"]
dataset_samsum_pt.set_format(type="torch", columns=columns)

A new thing in the use of tokenization step is the `tokenizer.as_target_tokenizer()` context. Some models require special tokens in the decoder inputs, so it's important to differentiate between the tokenization of encoder and decoder inputs. In the `with` statement (called a *context manager*), the tokenizer knows that it is tokenizing for the decoder and can process sequences accordingly.

Now, we need to create the data collator. This function is called in the `Trainer` just before the batch is fed through the model. In most cases we can use the default collator, which collects all the tensors from the batch and simply stacks them. For the summarization task we need to not only stack the inputs but also prepare the targets on the decoder side. 

PEGASUS is an encoder-decoder transformer and thus has the classic seq2seq architecture. In a seq2seq setup, a common approach is to apply teacher forcing in the decoder. With this strategy, the decoder receives input tokens that consists of the labels shifted by one in addition to the encoder output; so when making the prediction for the next token, the decoder gets the ground truth shifted by one as an input, as illustrated in the following table:

<img src="images/teacher_forcing_example.png" title="" alt="" width="450" data-align="center">

We shift it by one so that the decoder only sees the previous ground truth labels and not the current or future ones. Shifting alone suffices since the decoder has masked self-attention that masks all inputs at present and in the future. So, when we prepare our batch, we set up, the decoder inputs by shifting the labels to the right by one. After that, we make sure the padding tokens in the labels are ignored by the loss function by setting them to `-100`. We actually dont' have to this manyally, though, since the DataCollatorForSeq2Seq comes to the rescue and takes care of all these steps for us:

In [ ]:
from transformers import DataCollatorForSeq2Seq
from transformers import TrainingArguments, Trainer

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

training_args = TrainingArguments(
    output_dir='pegasus-samsum', 
    num_train_epochs=1, 
    warmup_steps=500,
    per_device_train_batch_size=1, 
    per_device_eval_batch_size=1,
    weight_decay=0.01, 
    logging_steps=10, 
    push_to_hub=False, # Set to false
    evaluation_strategy='steps', 
    eval_steps=500, 
    save_steps=1e6,
    gradient_accumulation_steps=16
)

One thing that is different from the previous settings is the `gradient_accumulation_steps` argument. Since the model is quite big, we had to set the batch size to 1. However, a batch size that is too small can hurt convergence. To resolve that issue, we can use a nifty technique called [gradient accumulation](https://huggingface.co/docs/accelerate/usage_guides/gradient_accumulation). As the name suggests, instead of calculating the gradients of the full batch at once, we make smaller batches and aggregate the gradients. When we have aggregated enough gradients we run the optimization step. Naturally this is a bit slower than doing it one pass, but **it saves us a log of GPU memory**.

We now have everything we need to initialize the trainer with:
* model
* tokenizer
* training arguments
* data collator
* training and evaluation sets

In [ ]:
trainer = Trainer(
    model=model, 
    args=training_args, 
    tokenizer=tokenizer, 
    data_collator=seq2seq_data_collator,
    train_dataset=dataset_samsum_pt["train"],
    eval_dataset=dataset_samsum_pt["validation"]
)

trainer.train()

After training, we can directly run the evaluation function on the test set to see how well the model performs:

In [ ]:
score = evaluate_summaries_pegasus(
    dataset_samsum["test"], 
    rouge_metric, 
    trainer.model, 
    tokenizer,
    batch_size=2, 
    column_text="dialogue", 
    column_summary="summary"
)

rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
pd.DataFrame(rouge_dict, index=[f"pegasus"])

We see that the ROUGE scores improved considerably over the model without finetuning, so even though the previous model was also trained for summarization, it was not well adapted for the new domain. 

----

**Note:** You can also evaluate the generations as part of the training loop: use the extension of `TrainingArguments` called `Seq2SeqTrainingArguments` and specify `predict_with_generate=True`. Pass it to the dedicated `Trainer` called `Seq2SeqTrainer`, which then uses the `generate()` function instead of the model's forward pass to create predictions for evaluation.

----

In [ ]:
# TODO: Try this idea

### 6.7.3 - Generating dialogue summaries

Looking at the losses and ROUGE scores, it seems the model is showing a significant improvement over the original model trained on CNN/DailyMail only. Let's see what a summary generated on a sample from the test set looks like:

In [ ]:
gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}
sample_text = dataset_samsum["test"][0]["dialogue"]
reference = dataset_samsum["test"][0]["summary"]
pipe = pipeline("summarization", model="transformersbook/pegasus-samsum")

print("Dialogue:")
print(sample_text)
print("\nReference Summary:")
print(reference)
print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

That looks much more like the reference summary. It seems the model has learned to synthesize the dialogue into a summary without just extracting passages. Now, the ultimate test: how well does the model work on a custom input?

In [ ]:
custom_dialogue = """\
Thom: Hi guys, have you heard of transformers?
Lewis: Yes, I used them recently!
Leandro: Indeed, there is a great library by Hugging Face.
Thom: I know, I helped build it ;)
Lewis: Cool, maybe we should write a book about it. What do you think?
Leandro: Great idea, how hard can it be?!
Thom: I am in!
Lewis: Awesome, let's do it together!
"""
print(pipe(custom_dialogue, **gen_kwargs)[0]["summary_text"])

## Conclusion

Text summarization poses some unique challenges compared to other tasks that can be framed as classification tasks, like sentiment analysis, named entity recognition, or question answering. Conventional metrics such as accuracy do not reflect the quality of the generated text. As we saw, the BLEU and ROUGE metrics can better evaluate generated texts; however, human judgment remains the best measure.

A common question when working with summarization models is how we can summarize documents where the texts are longer than the model's context length. Unfortunately, there is no single strategy to solve this problem, and to date this is still an open and active research question. For example, [recent work by OpenAI showed how to scale summarization by applying it recursively to long documents and using human feedback in the loop](https://arxiv.org/abs/2109.10862).

In the next chapter we'll look at question answering, which is the task of providing an answer to a question based on a text passage. In contrast to summarization, with this task there exist good strategies to deal with long or many documents, and we'll show how to scale question answering to thousands of documents.